In [1]:
!pip install requests
!pip install pydub

Chạy thử gen 1 audio

In [ ]:
import requests

# URL của API Zalo TTS
url = "https://api.zalo.ai/v1/tts/synthesize"

data = {
    "input": "Bật đèn 1",  # Văn bản cần chuyển đổi thành giọng nói
    "speaker_id": 3,  # ID giọng nói (có thể chọn theo giới tính, vùng miền, v.v.)
    "speed": 1.2  # Tốc độ giọng nói
}

headers = {
    # "apikey": "API_KEY"  # Thay your_api_key_here bằng API Key của bạn
}

response = requests.post(url, data=data, headers=headers)

# Kiểm tra xem request có thành công không
if response.status_code == 0:
    print(response.json())
else:
    print(f"Error: {response.status_code} - {response.text}")


print(response.json())

Error: 200 - {"data":{"url":"https://chunk-v3.tts.zalo.ai/secure/aab1bf8cd2fd3ba362ec/aab1bf8cd2fd3ba362ec.wav?expires=1747732888&md5=4AfonHDCyQuNKteGbdF9Zw"},"error_code":0,"error_message":"Successful."}
{'data': {'url': 'https://chunk-v3.tts.zalo.ai/secure/aab1bf8cd2fd3ba362ec/aab1bf8cd2fd3ba362ec.wav?expires=1747732888&md5=4AfonHDCyQuNKteGbdF9Zw'}, 'error_code': 0, 'error_message': 'Successful.'}


In [ ]:
import requests

url = 'https://api.fpt.ai/hmi/tts/v5'

payload = 'chào chị Vi'
headers = {
    'api-key': 'API_KEY',
    'speed': '3',
    'voice': 'banmai',
    'format': 'wav'
}

response = requests.request('POST', url, data=payload.encode('utf-8'), headers=headers)

print(response.text)


{"async":"https://file01.fpt.ai/text2speech-v5/short/2025-05-18/f355ae332baaafba9d884718ed2ebd27.wav","error":0,"message":"The content will be returned after a few seconds under the async link.","request_id":"68729329-8113-499b-a282-a7b31efa4874"}


In [ ]:
import requests
import os
import pandas as pd
from urllib.parse import urlparse
from pydub.utils import mediainfo
# Hàm để tải file âm thanh về từ URL
def download_audio(audio_url, filename):
    try:
        response = requests.get(audio_url, timeout=10)
        response.raise_for_status()  # Kiểm tra nếu có lỗi trong quá trình tải
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Audio downloaded: {filename}")
        return True
    except Exception as e:
        print(f"{e}")
        return False

# Hàm gọi API Zalo TTS và trả về URL của audio
def call_tts_api_zl(command_text, speaker_id, speed=1.2, api_key='API_KEY'):
    url = "https://api.zalo.ai/v1/tts/synthesize"
    data = {
        "input": command_text,
        "speaker_id": speaker_id,
        "speed": speed
    }
    headers = {
        "apikey": api_key
    }

    response = requests.post(url, data=data, headers=headers)

    if response.status_code == 200:
        audio_url = response.json().get("data").get("url")
        return audio_url
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def call_tts_api_fpt(command_text, speaker_id, speed='3', api_key='API_KEY'):
    url = "https://api.fpt.ai/hmi/tts/v5"

    headers = {
        "api-key": api_key,
        "speed": speed,
        "voice": speaker[speaker_id],
        "format": 'wav'
    }

    response = requests.post(url, data=command_text.encode('utf-8'), headers=headers)

    if response.status_code == 200:
        audio_url = response.json().get("async")
        return audio_url
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Hàm tính độ dài của file audio (dựa vào thời gian)
def get_audio_duration(filename):
    try:
        info = mediainfo(filename)
        duration = float(info['duration'])
        print(f"Duration of {filename}: {duration} seconds")
        return duration
    except Exception as e:
        print(f"Error getting duration for {filename}: {e}")
        return None

DATA_DIR = r"D:\Downloads\Đồ án_Voice\dataset_custom\vi"
CLIP_DURATIONS_PATH = os.path.join(DATA_DIR, "clip_durations.tsv")
VALIDATED_PATH = os.path.join(DATA_DIR, "validated.tsv")
print(CLIP_DURATIONS_PATH)
commands_df = pd.read_csv('commands.tsv', sep='\t')

gender = {1: 'female_feminine', 2: 'female_feminine', 3: 'male_masculine', 4: 'male_masculine',
          5: 'female_feminine', 6: 'male_masculine', 7: 'male_masculine', 8: 'male_masculine',
          9: 'female_feminine', 10: 'female_feminine', 11: 'female_feminine', 12: 'female_feminine'}
accents = {1: 'south', 2: 'northern', 3: 'south', 4: 'northern',
           5: 'central', 6: 'south', 7: 'central', 8: 'northerm',
           9: 'south', 10: 'south', 11: 'central', 12: 'northern'}

speaker = {5: 'myan', 6: 'minhquang', 7: 'giahuy', 8: 'leminh', 
           9: 'linhsan', 10: 'lannhi', 11: 'ngoclam', 12: 'banmai'}

speaker_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
audio_urls = []
command_locale = 'vi'


for si in speaker_id:
    for index, row in commands_df.iterrows():
        command = row['command']
        command_id = row['id']

        if si in [1, 2, 3, 4]:
            audio_url = call_tts_api_zl(command, speaker_id=si)
        else:
            audio_url = call_tts_api_fpt(command, speaker_id=si)
        # audio_url = response.json().get("data").get("url")
        # print(f"Audio URL: {audio_url}")
        if audio_url:
            audio_urls.append(audio_url)


print("Len Audio URLs:", len(audio_urls))
print(audio_urls)


D:\Downloads\Đồ án_Voice\dataset_custom\vi\clip_durations.tsv
Len Audio URLs: 9
['https://file01.fpt.ai/text2speech-v5/short/2025-05-18/66668bd41887ca6f45b2917eaa10896c.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/46b0b5524f336ee80cf216d94302c81f.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/b4652a372238626ade8fbb4e21c9257a.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/fee7b8a860f9766d72783ff85e4325f9.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/c7f797a1468b7afff94e329f9bd0319a.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/dabb68b4bb09f9aeb6d55ba9c46e47d9.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/0639e46a9f1a3015b29796cb2732e7d3.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/4a0e5c557b9e2de05f1be9658c0c5bb8.wav', 'https://file01.fpt.ai/text2speech-v5/short/2025-05-18/a2a4414fda324be6a65e046fcff1272b.wav']


In [ ]:
clip_duration_data = []
validated_data = []
chunk_size = 50
for i, si in enumerate(speaker_id):
    start = i * chunk_size
    end = start + chunk_size
    for index, audio_url in enumerate(audio_urls[start:]):
        # Tạo tên file cho audio
        filename = os.path.basename(urlparse(audio_url).path)
        audio_filename = os.path.join(DATA_DIR, 'clip', filename)
        check = download_audio(audio_url, audio_filename)
        print(f"Name: {filename}, Audio URL: {audio_url}")

        # Tải file âm thanh về
        if check:
            duration = get_audio_duration(audio_filename)
            
            if duration:
                clip_duration_data.append({'path': filename, 'duration[ms]': duration*1000})

                # Lưu thông tin vào validated_data
                validated_data.append({'client_id': si,
                                    'path': filename,
                                    'sentence_id': commands_df['id'][index],
                                    'sentence': commands_df['command'][index],
                                    'gender': gender[si],
                                    'accents': accents[si],
                                    'locale': command_locale,
                                    'url': audio_url
                                    })

print("Len Clip Duration Data:", len(clip_duration_data))
print("Len Validated Data:", len(validated_data))
print(validated_data)

Audio downloaded: D:\Downloads\Đồ án_Voice\dataset_custom\vi\clip\66668bd41887ca6f45b2917eaa10896c.wav
Name: 66668bd41887ca6f45b2917eaa10896c.wav, Audio URL: https://file01.fpt.ai/text2speech-v5/short/2025-05-18/66668bd41887ca6f45b2917eaa10896c.wav
Duration of D:\Downloads\Đồ án_Voice\dataset_custom\vi\clip\66668bd41887ca6f45b2917eaa10896c.wav: 0.801563 seconds
Audio downloaded: D:\Downloads\Đồ án_Voice\dataset_custom\vi\clip\46b0b5524f336ee80cf216d94302c81f.wav
Name: 46b0b5524f336ee80cf216d94302c81f.wav, Audio URL: https://file01.fpt.ai/text2speech-v5/short/2025-05-18/46b0b5524f336ee80cf216d94302c81f.wav
Duration of D:\Downloads\Đồ án_Voice\dataset_custom\vi\clip\46b0b5524f336ee80cf216d94302c81f.wav: 0.738563 seconds
Audio downloaded: D:\Downloads\Đồ án_Voice\dataset_custom\vi\clip\b4652a372238626ade8fbb4e21c9257a.wav
Name: b4652a372238626ade8fbb4e21c9257a.wav, Audio URL: https://file01.fpt.ai/text2speech-v5/short/2025-05-18/b4652a372238626ade8fbb4e21c9257a.wav
Duration of D:\Download

In [41]:
# Chuyển thông tin thành DataFrame và lưu vào các file TSV
clip_duration_df = pd.DataFrame(clip_duration_data)
clip_duration_df.to_csv(CLIP_DURATIONS_PATH, sep='\t', index=False, mode='a', header=False)

validated_df = pd.DataFrame(validated_data)
validated_df.to_csv(VALIDATED_PATH  , sep='\t', index=False, mode='a', header=False)

print("Process completed.")

Process completed.


In [ ]:
import csv

def sort_tsv_by_path(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        reader = list(csv.reader(f, delimiter='\t'))
        header = reader[0]
        rows = reader[1:]
    if input_file == 'D:/Downloads/Đồ án_Voice/dataset_custom/vi/clip_durations.tsv':
        rows.sort(key=lambda row: row[0])
    else:
        rows.sort(key=lambda row: row[1])

    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(header)
        writer.writerows(rows)

for file in os.listdir(DATA_DIR):
    if file.endswith('.tsv'):
        file_path = os.path.join(DATA_DIR, file)
        sort_tsv_by_path(file_path, file_path)